<a href="https://colab.research.google.com/github/gio2555/financial_sentiment_analysis_transformers/blob/main/financial_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install transformers to perform sentiment analysis
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
#tqdm progress bar in the for loop
from tqdm import tqdm
import requests
import json
import regex as re
import datetime

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

# Get stock articles of the last two months

### paid version of api news that loop on different querys and on more than 100 article

In [ ]:
api_key = "d4488f0d43c54bf791f9c28f26892935"

In [ ]:
#Set up the API request
api_key = "<Your NEWS API Key>"  # Replace with your News API key
base_url = "https://newsapi.org/v2/everything"
search_queries = ["baba", "baba stock", "alibaba"]  # Replace with your desired search queries
page_size = 100  # Number of articles per page

# Get the current date
#current_date = datetime.date.today().isoformat()

# Initialize variables
articles = []

# Pagination loop
for query in search_queries:
    page = 1
    while True:
        # Define the parameters for the API request
        params = {
            "apiKey": api_key,
            "q": query,
            "pageSize": page_size,
            "page": page
        }

        # Send the API request
        response = requests.get(base_url, params=params)
        data = response.json()

        # Process the response
        if response.status_code == 200:
            # Append articles to the list
            articles.extend(data["articles"])

            # Check if there are more pages
            total_pages = data["totalResults"] // page_size
            if page >= total_pages:
                break

            # Increment page number
            page += 1
        else:
            print(f"Error: {response.status_code} - {data['message']}")
            break

# Process the retrieved articles
for article in articles:
    # Extract relevant information from each article
    title = article["title"]
    description = article["description"]
    published_at = article["publishedAt"]

    # Do further processing or analysis with the retrieved news articles
    print(f"Title: {title}")
    print(f"Description: {description}")
    print(f"Published at: {published_at}")
    print("------------------------------------")


### free method with a 100 aricles scraper limit with news api

In [ ]:
import requests

api_key = "<Your NEWS API Key>"
query = "Alibaba stock"

url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}"
response = requests.get(url)

if response.status_code == 200:
    news_data = response.json()
    articles = news_data['articles']
    for article in articles:
        title = article['title']
        source = article['source']['name']
        published_at = article['publishedAt']
        print(f"Title: {title}\nSource: {source}\nPublished At: {published_at}\n")
else:
    print("Error occurred while fetching news.")


## Mediacloud

In [ ]:
!pip install mediacloud


  Preparing metadata (setup.py) ... done
  Created wheel for mediacloud: filename=mediacloud-4.0.1-py3-none-any.whl size=4943 sha256=f8794c84cc395f99bb3d3f7243fef00edd472dd28cfa226cfa5cb644ef7a9416
  Stored in directory: /root/.cache/pip/wheels/19/ff/76/9e686629a8dd89777a9d7e23e09eeb1b57a7ccf6362e0d45ac
Successfully built mediacloud


In [ ]:
import mediacloud.api
INDIA_NATIONAL_COLLECTION = 34412118
SOURCES_PER_PAGE = 100  # the number of sources retrieved per page
mc_directory = mediacloud.api.DirectoryApi('823162d7e4e886d8dec47e2fc3bf26a93a4359de')
sources = []
offset = 0   # offset for paging through
while True:
    # grab a page of sources in the collection
    response = mc_directory.source_list(collection_id=INDIA_NATIONAL_COLLECTION, limit=SOURCES_PER_PAGE, offset=offset)
    # add it to our running list of all the sources in the collection
    sources += response['results']
    # if there is no next page then we're done so bail out
    if response['next'] is None:
        break
    # otherwise setup to fetch the next page of sources
    offset += len(response['results'])
print("India National Collection has {} sources".format(len(sources)))

India National Collection has 136 sources


### mediacrawler

In [ ]:
!pip install comcrawl

In [ ]:
from comcrawl import IndexClient

client = IndexClient("2023-23")

client.search("reddit.com/r/MachineLearning/*")
client.download()

first_page_html = client.results[0]["html"]

In [ ]:
import requests

url = "https://newsapi.org/v2/top-headlines?sources=reuters&apiKey=<Your API Key>"
response = requests.get(url)

if response.status_code == 200:
    news_data = response.json()
    articles = news_data['articles']
    for article in articles:
        title = article['title']
        source = article['source']['name']
        published_at = article['publishedAt']
        print(f"Title: {title}\nSource: {source}\nPublished At: {published_at}\n")
else:
    print("Error occurred while fetching news.")


Error occurred while fetching news.


## perform data cleaning with pandas

In [ ]:
df = pd.DataFrame(articles)


In [ ]:
df = pd.DataFrame(data['articles']).drop('urlToImage', axis=1)

In [ ]:
df

,source,author,title,description,url,publishedAt,content
0,"{'id': None, 'name': 'ReadWrite'}",Aaron Heienickle,Alibaba’s Leadership Shake-Up: Major Changes U...,Chinese e-commerce behemoth Alibaba Group Hold...,https://readwrite.com/alibabas-leadership-shak...,2023-06-27T18:45:41Z,Chinese e-commerce behemoth Alibaba Group Hold...
1,"{'id': 'business-insider', 'name': 'Business I...",Huileng Tan,A Chinese EV billionaire's wealth has plummete...,"He Xiaopeng, the CEO of Xpeng, owns 20.5% of t...",https://www.businessinsider.com/ev-price-war-x...,2023-05-30T05:02:04Z,"He Xiaopeng, Chairman and CEO of Chinese elect..."
2,"{'id': None, 'name': 'MarketWatch'}",Emily Bary,The Ratings Game: Alibaba’s stock is ‘still ve...,Comparisons are about to get tougher for the e...,https://www.marketwatch.com/story/alibabas-sto...,2023-06-27T11:28:00Z,Alibaba Group Holding Ltd. shares might be a v...
3,"{'id': None, 'name': 'Yahoo Entertainment'}",Yahoo Finance,"PayPal-KKR deal, Alibaba shares fall, Eli Lill...",Alibaba stock falls after the company's CEO an...,https://finance.yahoo.com/video/paypal-kkr-dea...,2023-06-20T15:04:11Z,<ul><li>\r\n</li><li>\r\n</li><li>\r\n</li><li...
4,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (Neil Rozenbaum),Alibaba Stock: Headed to $100?,The risk-reward makes this very compelling.,https://www.fool.com/investing/2023/06/12/alib...,2023-06-12T15:00:00Z,"In this video, I will talk about Alibaba(BABA ..."
...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Q.ai - Powering a Personal Wealth Movement, Co...",Upside Foods And Good Meat Secure USDA Approva...,Investors are chomping at the bit to get in on...,https://www.forbes.com/sites/qai/2023/06/23/up...,2023-06-23T12:51:49Z,(Photo by Catherine LAI / AFP) / TO GO WITH As...
96,"{'id': 'independent', 'name': 'Independent'}",Holly Williams,Asos shares jump after reported £1bn bid from ...,The troubled online retailer was reportedly ap...,https://www.independent.co.uk/business/asos-sh...,2023-06-05T08:18:19Z,For free real time breaking news alerts sent s...
97,"{'id': None, 'name': 'Yahoo Entertainment'}",South China Morning Post,"Alibaba, Tencent, BYD drive Hong Kong stocks t...",Hong Kong stocks jumped by the most in two wee...,https://finance.yahoo.com/news/alibaba-tencent...,2023-06-15T09:30:00Z,Hong Kong stocks jumped by the most in two wee...
98,"{'id': None, 'name': 'MarketBeat'}",MarketBeat,A New Cycle For Alibaba Could Double Its Stock...,Alibaba Group (NYSE:BABA) has been increasing ...,https://www.marketbeat.com/originals/a-new-cyc...,2023-06-15T05:28:34Z,Key Points\r\n<ul><li>As the United States and...


In [ ]:
df['source'] = df['source'].astype(str)
df[['id', 'Magazine']] = df['source'].str.split(',', expand=True)
df = df.drop('source', axis=1).copy()
df = df.drop('id',axis=1).copy()

In [ ]:
df['Magazine'] = df['Magazine'].astype('string')

In [ ]:
df['Magazine']

0         'name': 'Business Insider'}
1              'name': 'MarketWatch'}
2      'name': 'Yahoo Entertainment'}
3              'name': 'Motley Fool'}
4              'name': 'Motley Fool'}
                   ...               
95              'name': 'Libsyn.com'}
96     'name': 'Yahoo Entertainment'}
97                  'name': 'Forbes'}
98             'name': 'Google News'}
99     'name': 'Yahoo Entertainment'}
Name: Magazine, Length: 100, dtype: string

In [ ]:
list_of_chars = ["'",'"', 'name', ':', ' ', '}']
pattern = '|'.join(map(re.escape, list_of_chars))
df['Magazine'] = df['Magazine'].str.replace(pattern, '')

<ipython-input-13-9f085a66f425>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Magazine'] = df['Magazine'].str.replace(pattern, '')


In [ ]:
df.rename(columns={'publishedAt':'date'}, inplace=True)

In [ ]:
df['date']= pd.to_datetime(df['date'])

In [ ]:
df['content'] = df['content'].astype('string')

# Sentiment analysis

## load the model

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

#### see if the transofromers work on an example

In [ ]:
#url of the article
df['url'][95]

'https://finance.yahoo.com/video/paypal-kkr-deal-alibaba-shares-150411977.html'

In [ ]:
example= df['content'][95]

In [ ]:
encoded_text = tokenizer(example,return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'negative': scores[0],
    'neutral': scores[1],
    'positive': scores[2]
}
print(scores_dict)

{'negative': 0.047906153, 'neutral': 0.90762335, 'positive': 0.044470493}


In [ ]:
max(scores_dict, key=scores_dict.get)

'neutral'

#### loop through all the database

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.rename(columns={'index':'id'})

In [ ]:
#loop
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {'negative': scores[0],
                   'neutral': scores[1],
                   'positive': scores[2]
    }
    return max(scores_dict, key=scores_dict.get)

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
  try:
      text = row['content']
      myid = row['index']
      roberta_result = polarity_scores_roberta(text)
      res[myid] = roberta_result
  except RuntimeError:
        print(f'Broke for id {myid}')


100%|██████████| 100/100 [00:24<00:00,  4.08it/s]


In [ ]:
res

In [ ]:
results_df = pd.DataFrame.from_dict(res, orient='index', columns=['sentiment'])
results_df = results_df.reset_index()
results_df = results_df.merge(df, how='left')

In [ ]:
results_df['url'][0]

'https://www.businessinsider.com/ev-price-war-xpeng-ceo-wealth-drop-billions-tesla-byd-2023-5'

#### Plot the sentiment analysis

In [ ]:
sentiment = results_df['sentiment'].value_counts().plot()

In [ ]:
sentiment['positive']

25

#different broker api

### offering different method broker app to perform the buying and selling of the stock

#### interactive broker api

In [ ]:
#the interactive broker api
!pip install ibapi         # Install the IB API library
!pip install ib_insync     # Install the ib_insync library (optional, but recommended for a higher-level interface)

In [ ]:
#the broker is gonna sell or buy the stock at the level at the price of the sentiment analysis
from ibapi import *
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.wrapper import EWrapper
from ibapi.order import Order

class IBWrapper(EWrapper):
    def __init__(self):
        self.orderId = None

    def nextValidId(self, orderId: int):
        self.orderId = orderId

    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 4:  # Last traded price
            # Place the order with the obtained current price
            app.place_order(instrument_id, action, order_type, quantity, price)

class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

class IBApp(IBWrapper, IBClient):
    def __init__(self):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)

        self.connect("127.0.0.1", 7497, clientId=1)  # Replace with appropriate connection details

    def place_order(self, instrument_id, action, order_type, quantity, price=None):
        contract = Contract()
        contract.symbol = instrument_id
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"

        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity
        if price:
            order.lmtPrice = price

        self.placeOrder(self.orderId, contract, order)

    def request_market_data(self, instrument_id):
        contract = Contract()
        contract.symbol = instrument_id
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"

        self.reqMktData(1, contract, "", False, False, [])

# Create an instance of the Interactive Brokers app
app = IBApp()

# Determine the order type based on sentiment
if sentiment['positive'] + sentiment['neutral'] > sentiment['negative']:
    # Execute a buy order
    instrument_id = 'BABA'  # Instrument ID or symbol of the security to buy
    action = "BUY"  # Order action
    order_type = "LMT"  # Order type
    quantity = 10  # Number of shares to buy

    # Request market data to obtain the current price
    app.request_market_data(instrument_id)
else:
    # Execute a sell order
    instrument_id = 'AAPL'  # Instrument ID or symbol of the security to sell
    action = "SELL"  # Order action
    order_type = "MKT"  # Order type
    quantity = 10  # Number of shares to sell

    app.place_order(instrument_id, action, order_type, quantity)

# Start the API message loop
app.run()

# Disconnect from the Interactive Brokers API
app.disconnect()


###unofficial webull app

In [ ]:
#unofficial api of webull https://github.com/tedchou12/webull
!pip install webull

In [ ]:
#the broker is gonna sell or buy the stock at the level at the price of the sentiment analysis
from webull import webull, endpoints

# Create a Webull API client
wb = webull()
wb.login("your_username", "your_password")

# Determine the order type based on sentiment
if sentiment['positive'] + sentiment['neutral'] > sentiment['negative']:
    # Execute a buy order
    instrument_id = 'BABA'  # Instrument ID or symbol of the security to buy
    order_type = 'MARKET'  # Order type
    quantity = 10  # Number of shares to buy

    # Get the current market price of the security
    quotes = wb.get_quote(instrument_id)
    current_price = quotes['lastTradePrice']

    wb.place_order(instrument_id, endpoints.TradeType.BUY, order_type, quantity, current_price)
else:
    # Execute a sell order
    instrument_id = 'BABA'  # Instrument ID or symbol of the security to sell
    order_type = 'MARKET'  # Order type
    quantity = 10  # Number of shares to sell

    wb.place_order(instrument_id, endpoints.TradeType.SELL, order_type, quantity)
